***

## Problem Set 8: Final Project

>CSC427, semester 202 (jan-may 2020)
<br>
burton rosenberg
<br>
univ of miami
<br>
(c) 2020 all rights reserved

<br>
_*Created: 20 April 2020*_
<br>
_*Last update: 21 April 2020*_

***



## Student's Name: Thanh Chau Do


### Statement of the project

In this final project, the study of P and NP will be practiced.

Your tasks are to,

1- Write a polynomial time predicate VerifyPath.find_if_path, that given a graph and two vertices,
   returns true if the two vertices are vertices in the graph, and there is a path in the graph connecting
   the two vertices. Else it returns False.

2- Write a polynomial time predicate VerifyKCover.verify, that given a graph and a list of vertices, 
   returns true if the list of vertices is a vertex-covering of the graph. That is, ever edge of the
   graph has at least one end vertice in the given cover. Else it returns False.
   
3- Write an exponential time search algoirthm, SolveKCover.search_brute_force, that given a graph and
   an integer k, returns a vertex-covering with exactly k vertices of the graph, or else returns the
   empty list. The algorithm will work by enumerating all subsets of k vertices from the set of all
   the graph's vertices, and will apply the verifying on each such candidate solution.
   
4- Write a polynomial time reduction Reduction.reduce from 3-SAT to Vertex Cover, that is, which given
   an instance of 3-SAT produces a graph and a number k, such that 3-SAT is satisfiable if and only
   if the graph has a k covering.
   
5- Write a polynomail time function Reduction.pullback, which given a 3-SAT to Vertex Cover reduction,
   and a k cover, produces a satisfying assignment for the 3-SAT instance, the subject of the reduction.
   
6- Put it all together: Write a 3-SAT solver that solves a 3-SAT instance by reducing it to a Vertex Cover
   instance, solves the instance by brute force, and pulls back the vertex cover solution to a satisfying 
   assignment for the 3-SAT instance.
   

   
   

#### Description of data structure

__Description of a graph__

1- A graph is made up of vertices and edges. Each edge is a pair of vertices

2- A vertex will be represented as a string, to be thought of as the name of the vertex.

3- An edge will be represented as a Python pair. The two members of the pair will be the vertex name.

4- A graph will be a Python list of pairs.


In [388]:
# Example of a graph. The textbook graph of Figure 0.12 (b) is represented -

figure_0_12_b = [
    ('1','2'),
    ('1','3'),
    ('1','4'),
    ('2','3'),
    ('2','4'),
    ('3','4')
]


__Description of a 3-SAT__

1- A 3-SAT is made up of clauses, signed variables, and variables.

2- A variable will be represented as a string, to be thought of as the name of the variable.

3- A signed variable is a Python pair. The first element is a variable name and the second
   element is 0 or 1. 
 * If 0, the sign of the variable is positive, the variable is appearing
   without negation; 
 * If 1, the sign of the variable is negative, the variable is appearing
   negated.

4- A clause is Python list of 3 signed variables.

5- An instance of 3-SAT is a Python list of clauses.


In [389]:
# The 3-SAT in Figure 7.33 is represented - 

figure_7_45 = [
    [('x1',0),('x1',0),('x2',0)],
    [('x1',1),('x2',1),('x2',1)],
    [('x1',1),('x2',0),('x2',0)]
]


__Description of an instance of k-VC__

An instance of a k vertex cover problem is a Python pair.

   * The first element of the pair is the integer k
   * The second element of the pair is a graph

***

## Student workspace



### Problem 1: P-Time Path algorithm

In [390]:

class VerifyPath:
    @staticmethod
    def get_adjacent_edges(graph, source):
        #print("source: ", source)
        result = []
        
        for g in graph:
            if source in g:
                for el in g:
                    if el != source:
                        result.append(el)
        #print("adjacent nodes: ", result)
        return result
    
    @staticmethod
    def find_if_path(graph,source,dest):

        pass

        # data structures:
        #   visited, a set of visited vertices 
        #   edge_dict, a dictionary from a vertex to a list of neighbors
        #   q, a list of unvisited vertices, known to be reachable 
        #      from source, to be visited
        
        visited = set([source])
        #edge_dict = {source:VerifyPath.get_adjacent_edges(graph, source)}   # should be a list of neighbors of source
        #q = edge_dict[source]
        q = [source]
        
        while len(q)>0:
            v = q.pop(0)
            #print("v: ", v)
            #print("q: ", q)
            #print("visited: ", visited)
            #visited.add(v)
            
            if v == dest:
                break
            else:
                for w in VerifyPath.get_adjacent_edges(graph, v):
                    #print("w: ", w)
                    if w not in visited:
                        visited.add(w)
                        q.append(w)
    
        return dest in visited


### Problem 2: P-Time Vertex Cover verifier

In [391]:
class VerifyKCover:
    
    def __init__(self,graph,verbose=set()):
        self.graph = graph
        self.verbose = verbose

    def verify(self,cover):
        """
        verify that the vertex list given is a k-cover
        """
        
        # List<Edge>
        # Edge = (node, node)
        # Input is a possible vertex cover which is just a list of nodes List<Node>
        # Check if each edge in List<Edge> contains a node in List<Node>, 
        #  then cover is a vertex cover.
        
        for g in self.graph:
            #print(g)
            if g[0] not in cover and g[1] not in cover:
                #print("Not in cover")
                return False
    
        return True


### Problem 3: Exp-Time Vertex Cover solver

In [392]:
import itertools

class SolveKCover:
    
    def __init__(self,graph,verbose=set()):
        self.graph = graph
        self.verbose = verbose
        self.verifier = VerifyKCover(graph,verbose)
        
        self.iterate_idx = 0
        self.k = 0
        
        # make a list of vertices of the graph
        v = set()
        for edge in graph:
            v.add(edge[0])
            v.add(edge[1])
        self.vertices = list(v)
       
    def get_vertices(self):
        return self.vertices
    
    def get_verifier(self):
        return self.verifier
    
    def get_ith_bit(self,i,n):
        return (n//2**i)%2

    def count_ones(self,n):
        count = 0
        while n>0:
            count += n%2
            n = n//2
        return count

    def new_iteration(self,k):
        self.k = k
        self.iterate_idx = 0
        
    def next_iterate(self):
        idx = self.iterate_idx + 1
        while (self.count_ones(idx)!=self.k):
            idx += 1
        self.iterate_idx = idx
        return idx

    def select_vertices_by_index(self,idx):
        l = len(self.vertices)
        i = 0
        selected = []
        while idx>0 and i<l:
            if idx%2 == 1:
                selected.append(self.vertices[i])   
            idx = idx//2
            i +=1
        return selected

    def get_subsets_k(self, k):
        s = self.get_vertices()
        return list(itertools.combinations(s, k))   
    
    def search_brute_force(self,k):
        
        v = VerifyKCover(self.graph)
        cover = self.get_subsets_k(k)

        n_vertices = len(self.vertices)
        
        if k>n_vertices:
            return []
        if k<=0:
            return []

        assert(k>0)

        for c in cover:
            if v.verify(list(c)):
                return list(c)
            
        return []



In [393]:
# Exercise: Find the minimum length vertex cover of a graph
import random

def get_min_cover(graph):
    result = set()
    vertices = set()
    for edge in graph:
        vertices.add(edge[0])
        vertices.add(edge[1])

    while len(graph) != 0:
        
        v = vertices.pop()
        #print("v: ", v)
        result.add(v)
        temp = graph.copy() # temp graph, to avoid error in list iteration
                
        for neigh in graph:
            #print("neigh: ", neigh)
            if neigh[0] == v or neigh[1] == v:
                #print("\tremoving: ", neigh)
                temp.remove(neigh)
                vertices.discard(neigh[0])
                vertices.discard(neigh[1])
                
        graph = temp
        #print("graph2: ", graph)
    
    return result

### Some examples of how to use the enumerating functions

In [394]:
# using the functions implemented in SOlveKCover to
# enumerate the integers with binary representation 
# containing exact one 1 bit. e.g. 1, 2, 4, ...
                  
def iterator_pure_powers(n_max):
    skc = SolveKCover([('a','a')])
    skc.new_iteration(1)
    idx = skc.next_iterate()
    while idx<n_max:
        print(idx)
        idx = skc.next_iterate()

iterator_pure_powers(5000)

# using the functions implemented in SOlveKCover to
# enumerate all subsets of 3 items from a set of n items


def enumerate_3_sets(n):
    
    graph = []
    for i in range(n):
        var = 'var_{}'.format(i)
        graph.append((var,var))

    skc = SolveKCover(graph)
    skc.new_iteration(3)
    idx = skc.next_iterate()
    idx_max = 2**len(graph)
    print('number of s/sets = {}'.format(int(n*(n-1)*(n-2)/6)))
    i = 1
    while idx<idx_max:
        print('{}:\t{}'.format(i,skc.select_vertices_by_index(idx)))
        idx = skc.next_iterate()
        i += 1

                      
enumerate_3_sets(6)


1
2
4
8
16
32
64
128
256
512
1024
2048
4096
number of s/sets = 20
1:	['var_4', 'var_2', 'var_3']
2:	['var_4', 'var_2', 'var_1']
3:	['var_4', 'var_3', 'var_1']
4:	['var_2', 'var_3', 'var_1']
5:	['var_4', 'var_2', 'var_0']
6:	['var_4', 'var_3', 'var_0']
7:	['var_2', 'var_3', 'var_0']
8:	['var_4', 'var_1', 'var_0']
9:	['var_2', 'var_1', 'var_0']
10:	['var_3', 'var_1', 'var_0']
11:	['var_4', 'var_2', 'var_5']
12:	['var_4', 'var_3', 'var_5']
13:	['var_2', 'var_3', 'var_5']
14:	['var_4', 'var_1', 'var_5']
15:	['var_2', 'var_1', 'var_5']
16:	['var_3', 'var_1', 'var_5']
17:	['var_4', 'var_0', 'var_5']
18:	['var_2', 'var_0', 'var_5']
19:	['var_3', 'var_0', 'var_5']
20:	['var_1', 'var_0', 'var_5']


### Problem 4: Reducing 3-SAT to k-VC

In [395]:
import re
import itertools

class Reduction:

    def __init_(self,verbose=set()):
        self.verbose = verbose

    def get_all_var(target):
        all_var = set()
        for clause in target:
            for var in clause:
                all_var.add(var[0])      
                
        return all_var
        
        
    def get_K2(all_var):
        
        K_2 = []
        K_2_vc = []
        
        for v in all_var:
            #K_2.append((v, 0))  
            #K_2.append((v, 1)) 
            K_2.append(v+'_0_K2')
            K_2.append(v+'_1_K2')
            K_2_vc.append((v+'_0_K2',v+'_1_K2'))
            
        return [K_2, K_2_vc]
    

    def get_edge_C3(C_3, count):
        
        """"
         Sorry for the lengthy notation, but in order to make each node unique, 
         and for the output graph to be compatiable with the functions in Q1, Q2, Q3, 
         I have adopted the following notation:
         For nodes in the clause gadget (C3):
          x1_0_C3_1_1 means:
            variable: x1
            negative or positive: 0 - negative
            C3: it is a gadget variable (in the clause, C3)
            1: it is in the 1st clause
            1: it is the 1st variable in the 1st clause
        """
        
        vc1 = []
        vc2 = []

        vc1.append((str(C_3[0][0])+'_'+str(C_3[0][1])+'_C3_'+str(count)+'_1', 
                    str(C_3[1][0])+'_'+str(C_3[1][1])+'_C3_'+str(count)+'_2'))
        vc1.append((str(C_3[0][0])+'_'+str(C_3[0][1])+'_C3_'+str(count)+'_1', 
                    str(C_3[2][0])+'_'+str(C_3[2][1])+'_C3_'+str(count)+'_3'))
        vc1.append((str(C_3[1][0])+'_'+str(C_3[1][1])+'_C3_'+str(count)+'_2', 
                    str(C_3[2][0])+'_'+str(C_3[2][1])+'_C3_'+str(count)+'_3'))
        vc2.append((str(C_3[0][0])+'_'+str(C_3[0][1])+'_C3_'+str(count)+'_1',
                    str(C_3[0][0])+'_'+str(C_3[0][1])+'_K2'))
        vc2.append((str(C_3[1][0])+'_'+str(C_3[1][1])+'_C3_'+str(count)+'_2', 
                    str(C_3[1][0])+'_'+str(C_3[1][1])+'_K2'))
        vc2.append((str(C_3[2][0])+'_'+str(C_3[2][1])+'_C3_'+str(count)+'_3', 
                    str(C_3[2][0])+'_'+str(C_3[2][1])+'_K2'))
        #print("C_3: ", C_3)
        #print("vc1: ", vc1)
        #print("vc2: ", vc2)
        return vc1 + vc2        
    
    
    def get_cartesian_K2(vc):
        # get variables in K2 with TRUE Boolean assignment
        K2_pair = set()
        for edge in vc:
            for node in edge:
                if '_0_K2' in node:
                    K2_pair.add((node, node[0:node.find('_0_K2')]+'_1_K2'))
                    
        #print("K2 pair: ", K2_pair)
        # Create a list of pair set of positive and negative
        # Then create a list of the Cartesian product of of the sets
        
        K2_cartesian = []
        
        for tup in itertools.product(*K2_pair):
            #print(tup)
            K2_cartesian.append(tup)
        
        return K2_cartesian
    
    
    def count_clauses(vc):
        max = -1
        
        for edge in vc:
            for v in edge:
                if not 'K2' in v:
                    start = v.find('_C3_') + len('_C3_')
                    end = v.find('_', start)
                    num = v[start:end]
                    num = int(num)
                    if (num > max):
                        max = num
        return max
    
    
    @staticmethod
    def reduce(instance_3sat):
        """
        The widget: create C_3 for every clause, and K_2 for each variable.
        Associate each of the vertices in the C_3 with a variable in the clause, and the
        pair of vertices in the K_2 with the postive and negated forms of the variable.
        
        Also create an edge from the vertex in C_3 for variable v (resp not v) to the
        vertices in K_2 for variable v (resp not v)
        
        Any cover must be at least n+2m, for n variables and m clauses. The cover sought has k = m+2l.
        Where:
            k = The number of nodes in the vertex cover
            m = number of variables
            l = number of clauses
        """
        
        # collect up the nodes
        all_var = Reduction.get_all_var(instance_3sat)           
        temp = Reduction.get_K2(all_var)   
        
        K_2 = temp[0]
        instance_vc = temp[1]
        #print("K_2: ", K_2)
        #print("instance_vc:\n ", instance_vc)
        
        count = 0        
        for clause in instance_3sat:
            count += 1
            instance_vc = instance_vc + Reduction.get_edge_C3(clause, count)
                    
        k = len(all_var) + 2*len(instance_3sat)
                
        return (k, instance_vc)
    
    
    @staticmethod
    def pullback(vertex_cover):
        assignment = {}
        res_gadget = set()
        
        # The vertex cover must contain one node in each variable gadget
        # So we return a list of the Cartesian sets of the variable gadgets
        K2_cartesian = Reduction.get_cartesian_K2(vertex_cover) 
        l = Reduction.count_clauses(vertex_cover)
        m = len(K2_cartesian[0])  # total number of variables
        k = m + 2*l
        
        # print("K2: ", K2_cartesian)
        # print("m: ", m, " - l: ", l, " - k: ", k)
        
        # Try with each Cartesian tuple
        #  assign True for every variable in the tuple
        #  get all adjacent edges        
        for var_gadget in K2_cartesian:
            temp = set()
            
            for var in var_gadget:
                adj_v1 = VerifyPath.get_adjacent_edges(vertex_cover, var)
                
                for v1 in adj_v1:
                    if 'K2' not in v1:
                        temp.add(v1)
                        adj_v2 = VerifyPath.get_adjacent_edges(vertex_cover, v1)
                        
                        for v2 in adj_v2:
                                if 'K2' not in v2:
                                    temp.add(v2)    
                                    
            if(len(temp)/3 == l):
                #print(var_gadget, ": ", len(temp), ": ", sorted(temp))
                res_gadget = set(var_gadget)
                break;
        
        #print("res_gadget: ", res_gadget)
        
        for var in res_gadget:
            end1 = var.find('_')
            symbol = var[0:end1]
            start2 = end1+1
            end2 = var.find('_K2')
            sign = var[start2:end2]
            if sign == '1':
                assignment[symbol] = False
            elif sign == '0':
                assignment[symbol] = True
            else:
                print("ERROR in String Processing")
        
        return assignment
    
    @staticmethod
    def solve(instance_3sat):
        vertex_cover = Reduction.reduce(instance_3sat)[1] 
        assignment = Reduction.pullback(vertex_cover)
        return assignment  


***

## Some example tests

#### Problem 1: P-Time Path algorithm

In [397]:
graph = [('a','b'),('b','c'),('c','a'),('d','e')]
print("1: ", VerifyPath.find_if_path(graph, 'a', 'b')) # True
print("2: ", VerifyPath.find_if_path(graph, 'a', 'e')) # False   
print("3: ", VerifyPath.find_if_path(graph, 'd', 'e')) # True
print("4: ", VerifyPath.find_if_path(graph, 'd', 'c')) # False

print()    
graph1 = [(0,1), (0,4), (1,2), (1,3), (2,3), (3,4)]    
print("5: ", VerifyPath.find_if_path(graph1, 0, 2))    # True 
print("6: ", VerifyPath.find_if_path(graph1, 1, 4))    # True

print()
graph2 = [(0,1), (1,2), (2,3), (3,4), (4,5), (5,6), (8,9)]
print("8: ", VerifyPath.find_if_path(graph2, 0, 6))     # True
print("9: ", VerifyPath.find_if_path(graph2, 0, 9))     # False


1:  True
2:  False
3:  True
4:  False

5:  True
6:  True

8:  True
9:  False


#### Problem 2: P-Time Vertex Cover verifier

In [398]:
g1 = [('a','b'),('b','c'),('c','d'),('d','a'),('d','e'),('e','f'),('f','g1'),('f','g2')]
v = VerifyKCover(g1)
print("1: ", v.verify(['b','d','f'])) # true
print("2: ", v.verify(['g1','g2','e','a','c'])) # true 
print("3: ", v.verify(['a', 'b', 'c', 'd', 'e', 'f', 'g1', 'g2'])) # true
print("4: ", v.verify(['a', 'g2', 'b']))  # false
print("5: ", v.verify(['f', 'e', 'b', 'g2', 'c']))  # false

print()
g2 = [('a', 'b'), ('a', 'd'), ('b', 'c'), ('b', 'd'), ('b', 'e'), 
      ('c', 'e'), ('c', 'f'), ('d', 'e'), ('e', 'f')]
v2 = VerifyKCover(g2)
print("5: ", v2.verify(['b', 'c', 'd', 'f', 'a'])) # true
print("6: ", v2.verify(['b', 'c', 'd', 'f'])) # true
print("7: ", v2.verify(['b', 'c', 'd'])) # false


1:  True
2:  True
3:  True
4:  False
5:  False

5:  True
6:  True
7:  False


#### Problem 3: Exp-Time Vertex Cover

In [399]:
g1 = [('a','b'),('b','c'),('c','d'),('d','a'),('d','e'),('e','f'),('f','g1'),('f','g2')]
s = SolveKCover(g1)
for i in range (0, 10):
    solution = s.search_brute_force(i)
    print("k=", i,": ", solution)
    if len(solution)>0:
        print(s.get_verifier().verify(solution))

k= 0 :  []
k= 1 :  []
k= 2 :  []
k= 3 :  ['f', 'b', 'd']
True
k= 4 :  ['g2', 'f', 'b', 'd']
True
k= 5 :  ['g2', 'e', 'c', 'f', 'a']
True
k= 6 :  ['g2', 'e', 'c', 'f', 'b', 'd']
True
k= 7 :  ['g2', 'e', 'c', 'f', 'b', 'd', 'a']
True
k= 8 :  ['g2', 'e', 'c', 'f', 'b', 'd', 'a', 'g1']
True
k= 9 :  []


In [400]:
g2 = [('a', 'b'), ('b', 'c'), ('b', 'd'), ('c', 'e'), ('c', 'f')]
s2 = SolveKCover(g2)
for i in range(0, 8):
    solution = s2.search_brute_force(i)
    print("k=", i, ": ", solution)
    if len(solution)>0:
        print(s2.get_verifier().verify(solution))

k= 0 :  []
k= 1 :  []
k= 2 :  ['c', 'b']
True
k= 3 :  ['e', 'c', 'b']
True
k= 4 :  ['e', 'c', 'f', 'b']
True
k= 5 :  ['e', 'c', 'f', 'b', 'd']
True
k= 6 :  ['e', 'c', 'f', 'b', 'd', 'a']
True
k= 7 :  []


#### Additional Basic Test

In [401]:
def verify_path_tests(graph,tests):
    print("\n*** begin verify_path_tests ***")
    for test in tests:
        if VerifyPath.find_if_path(graph, test[1], test[2]) != test[0]:
            print('\tBAD: ({},{}) gives {}'.format(test[1],test[2],test[0]))
            print("*** fails verify_path_tests ***\n")
            return False
        print('\tOK: ({},{}) gives {}'.format(test[1],test[2],test[0]))
    print("*** passes verify_path_tests ***\n")
    return True


def verity_k_cover_tests(graph,tests):
    print("\n*** begin verify k cover tests ***")
    v = VerifyKCover(graph)
    for test in tests:
        if v.verify(test[1]) != test[0]:
            print('\tBAD: {} gives {}'.format(test[1],test[0]))
            print("*** fails verify k cover tests ***\n")
            return False
        print('\tOK: {} gives {}'.format(test[1],test[0]))
    print("*** passes verify k cover tests ***\n")
    return True


def solve_k_cover_example(graph,ks):
    print("\n*** begin SolveKCover tests ***")
    s = SolveKCover(graph)
    for k in ks:
        solution = s.search_brute_force(k[1])
        if k[0]:
            if (len(solution)>0) and not s.get_verifier().verify(solution):
                print('\tBAD: {}'.format(k[1]))
                print("*** fails SolveKCover tests ***\n")
                return False
            if len(solution)==0:
                print('\tBAD: {}'.format(k[1]))
                print("*** fails SolveKCover tests ***\n")
                return False                
        if not k[0] and (len(solution)>0):
            print('\tBAD: {}'.format(k[1]))
            print("*** fails SolveKCover tests ***\n")
            return False
        print('\tOK: {}'.format(k[1]))
    print("*** passes SolveKCover tests ***\n")
    return True


gr1 = [('a','b'),('b','c'),('c','a'),('d','e')]
gr1_answers = [[True,'a','b'], [False,'a','e']]

gr2 = [('a','b1'),('b1','c1'),('a','b2'),('b2','c2'),('a','b3'),('b3','c3'),('d','e'),('e','f')]
gr2_answers = [[True,'c3','c2'],[True,'b1','b2'],[False,'b1','e'],[True,'d','f']]

gr3 = [('a','b'),('b','c'),('c','d'),('d','a'),('d','e'),('e','f'),('f','g1'),('f','g2')]
gr3_answers = [[True,['b','d','f']],[True,['g1','g2','e','a','c']],[False,['c','e','g2']]]
gr3_k_solvable = [[False,1],[False,2],[True,3],[True,4],[True,5]]

verify_path_tests(gr1,gr1_answers)
verify_path_tests(gr2,gr2_answers)
verity_k_cover_tests(gr3,gr3_answers)
solve_k_cover_example(gr3,gr3_k_solvable)


*** begin verify_path_tests ***
	OK: (a,b) gives True
	OK: (a,e) gives False
*** passes verify_path_tests ***


*** begin verify_path_tests ***
	OK: (c3,c2) gives True
	OK: (b1,b2) gives True
	OK: (b1,e) gives False
	OK: (d,f) gives True
*** passes verify_path_tests ***


*** begin verify k cover tests ***
	OK: ['b', 'd', 'f'] gives True
	OK: ['g1', 'g2', 'e', 'a', 'c'] gives True
	OK: ['c', 'e', 'g2'] gives False
*** passes verify k cover tests ***


*** begin SolveKCover tests ***
	OK: 1
	OK: 2
	OK: 3
	OK: 4
	OK: 5
*** passes SolveKCover tests ***



True

### Problem 4 - 6: 3SAT and Vertex Cover Reduction

In [404]:
figure_7_45 = [
    [('x1',0),('x1',0),('x2',0)],
    [('x1',1),('x2',1),('x2',1)],
    [('x1',1),('x2',0),('x2',0)]
]
Reduction.reduce(figure_7_45)
print("figure_7_45:")
print(Reduction.solve(figure_7_45))
print()

sat_1 = [
    [('a',0),('b',0),('c',0)]
]
print("sat_1:")
print(Reduction.solve(sat_1))
print()

sat_2= [
    [('a',1),('a',1),('a',1)]
]
print("sat_2:")
print(Reduction.solve(sat_2))
print()

sat_3 = [
    [('a',0),('b',0),('c',0)],
    [('a',1),('b',1),('c',1)]
]
print("sat_3:")
print(Reduction.solve(sat_3))
print()

# An unsatisfiable 3-SAT formula
sat_4 = [
    [('x',0),('x',0),('x',0)],
    [('x',1),('x',1),('x',1)]
]
print("sat_4:")
print(Reduction.solve(sat_4))
print()

sat_5 = [
    [('x',0),('y',0),('z',0)],
    [('x',0),('y',0),('z',1)],
    [('x',0),('y',1),('z',0)],
    [('x',0),('y',1),('z',1)],
    [('x',1),('y',0),('z',0)],
    [('x',1),('y',0),('z',1)],
    [('x',1),('y',1),('z',0)],
    [('x',1),('y',1),('z',1)]
]
print("sat_5:")
print(Reduction.solve(sat_5))
print()


figure_7_45:
{'x1': False, 'x2': True}

sat_1:
{'a': True, 'b': True, 'c': True}

sat_2:
{'a': False}

sat_3:
{'a': True, 'b': False, 'c': True}

sat_4:
{}

sat_5:
{}

